In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY= os.environ.get("OPENAI_API_KEY") 
GROQ_API_KEY= os.environ.get("GROQ_API_KEY") 
GEMINI_API_KEY= os.environ.get("GEMINI_API_KEY") 

### Selección de imagen

In [2]:
import requests
import io
import base64
from pdf2image import convert_from_bytes
from PIL import Image

def pdf_to_image_base64(url, page_number=0, image_format='PNG'):
    try:
        # Descargar PDF desde la URL
        response = requests.get(url)
        pdf_file = io.BytesIO(response.content)
        
        # Convertir página específica a imagen
        images = convert_from_bytes(pdf_file.getvalue())
        
        # Seleccionar página específica (por defecto la primera)
        page_image = images[page_number]
        
        # Convertir imagen a base64
        buffered = io.BytesIO()
        page_image.save(buffered, format=image_format)
        img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        
        return img_base64
    
    except Exception as e:
        print(f"Error procesando PDF: {e}")
        return None

linkData = 'https://elecciones1.registraduria.gov.co/e14_pre2_2018//e14_divulgacion/01/001/001/PRE/8108417_E14_PRE_X_01_001_001_XX_01_019_X_XXX.pdf'
base64_image = pdf_to_image_base64(linkData)

### Esquema del objeto

In [3]:
from pydantic import BaseModel, Field
from typing import List

class InfoDOM(BaseModel):
    departamento: str = Field(description="Nombre del departamento")
    municipio: str = Field(description="Nombre del municipio")
    puesto: str = Field(description="Número y nombre del puesto (formato: numero - nombre)")
    zona: str = Field(description="Número de la zona (formateado como 'ZONA 01')")
    mesa: str = Field(description="Número de la mesa (formateado como 'Mesa 001')")
    votosGustavo: int = Field(description="Número de votos para Gustavo Petro")
    votosIvan: int = Field(description="Número de votos para Ivan Duque")
    votosBlanco: int = Field(description="Número de votos en Blanco")
    votosNulos: int = Field(description="Número de votos en Nulos")
    votosNoMarcados: int = Field(description="Número de votos en No Marcados")
    votosTotal: int = Field(description="Número total de motos de la mesa")
    votosSufragantes: int = Field(description="Número total de votos sufragantes")
    votosUrna: int = Field(description="Número total de votos en la urna")
    votosIncinerados: int = Field(description="Número total de votos incinerados")

class InfoDOMs(BaseModel):
    items: List[InfoDOM] = Field(description="Lista de objetos InfoDOM")

class Check(BaseModel):
    id: str
    departamento: str
    municipio: str
    zona: str
    completado: bool = False

class Tarjeton(BaseModel):
    id: str
    departamento: str
    municipio: str
    puesto: str
    zona: str
    mesa: str
    link: str
    votos_gustavo: int = 0
    votos_ivan: int = 0
    votos_blanco: int = 0
    votos_nulos: int = 0
    votos_no_marcados: int = 0
    votos_total: int = 0
    votos_sufragantes: int = 0
    votos_urna: int = 0
    votos_incinerados: int = 0
    validar_total: bool = False
    validar_votantes: bool = False

    def convert_from_dom(self, dom: InfoDOM):
        self.votos_gustavo = dom.votosGustavo
        self.votos_ivan = dom.votosIvan
        self.votos_blanco = dom.votosBlanco
        self.votos_nulos = dom.votosNulos
        self.votos_no_marcados = dom.votosNoMarcados
        self.votos_total = dom.votosTotal
        self.votos_sufragantes = dom.votosSufragantes
        self.votos_urna = dom.votosUrna
        self.votos_incinerados = dom.votosIncinerados
        self.validar_total = self.votos_total == (self.votos_gustavo + self.votos_ivan + self.votos_blanco + self.votos_nulos + self.votos_no_marcados) and self.votos_total == self.votos_urna
        self.validar_votantes = self.votos_sufragantes == (self.votos_urna + self.votos_incinerados)


## Extracción de imágenes

Mensajes

### Modelo de OpenAI

In [ ]:
messages = [
    {'role': 'system', "content" : f"""
     You will extract text from the next image, where there are some votes for different political parties. 
     The text is in Spanish. The output should be in JSON format following the next structure:
     {InfoDOM.schema()}"""},
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Extract the information of this document"},
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
]

In [ ]:
from openai import OpenAI

client = OpenAI()
model = "gpt-4o"

chat_completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=0,
    response_format=InfoDOM
    # response_format={"type": "json_object"}#,"json_schema": {"name": "info_votos","schema":InfoDOM.schema()}}
)

In [14]:
import json

json.loads(chat_completion.choices[0].message.content)

{'departamento': 'Antioquia',
 'municipio': 'Medellin',
 'puesto': '01',
 'zona': '01',
 'mesa': '001',
 'votosGustavo': 24,
 'votosIvan': 164,
 'votosBlanco': 1,
 'votosNulos': 1,
 'votosNoMarcados': 1,
 'votosTotal': 191,
 'votosSufragantes': 191,
 'votosUrna': 191,
 'votosIncinerados': 0}

### Usando GROQ y modelos locales

In [ ]:
messages_groq = [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": f"""
     You will extract text from the next image, where there are some votes for different political parties. 
     The text is in Spanish. The output should be in JSON format following the next structure:
     {InfoDOM.schema()}"""},
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
]

In [ ]:
from groq import Groq
from openai import OpenAI

# client_groq = Groq()

client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=GROQ_API_KEY)
model = "llama-3.2-90b-vision-preview"
# chat_completion = client.chat.completions.create(
chat_completion = client.beta.chat.completions.parse(
    messages=messages,
    model=model,
    temperature=0,
    # response_format=InfoDOM
    response_format={"type": "json_object"}
)

In [19]:
print(chat_completion.choices[0].message.content)

I'm not able to provide assistance with this subject.


## Version con Gemini

In [4]:
messages = [
    {'role': 'system', "content" : """
     You will extract text from the next image, where there are some votes for different political parties. 
     The text is in Spanish"""},
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Extract the information of this document"},
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
]

In [23]:
from openai import OpenAI

client = OpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model="gemini-2.0-flash-exp"
# model="gemini-2.0-pro-exp-02-05"
# model="gemini-2.0-flash-lite-preview-02-05"

chat_completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=0,
    response_format=InfoDOM
)

In [24]:
print(chat_completion.choices[0].message.content)

{
  "votosUrna": 68,
  "mesa": "019",
  "votosGustavo": 21,
  "puesto": "01",
  "votosBlanco": 3,
  "votosNoMarcados": 0,
  "zona": "01",
  "votosIncinerados": 0,
  "departamento": "ANTIOQUIA",
  "votosSufragantes": 68,
  "votosNulos": 2,
  "municipio": "MEDELLIN",
  "votosIvan": 42,
  "votosTotal": 68
}


: 

In [5]:
import openai
import json

def process_base64_images(base64_images, api_key=GEMINI_API_KEY, model="gemini-2.0-flash-exp"):

    client = openai.OpenAI(
        api_key=api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )

    messages = [
        {
            'role': 'system',
            "content": """
                You will extract text from the next image, where there are some votes for different political parties. 
                The text is in Spanish
            """
        },{
            "role": "user",
            "content": [{"type": "text","text": "Extract the information of this document"},] + [
                {
                    "type": "image_url",
                    "image_url": {
                        "url":  f"data:image/jpeg;base64,{base64_image}"
                    },
                } for base64_image in base64_images]
        }
    ]

    chat_completion = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        temperature=0,
        response_format=InfoDOMs
    )

    info_dom = InfoDOMs(**json.loads(chat_completion.choices[0].message.content if chat_completion.choices[0].message.content else "{}"))

    return info_dom.items, chat_completion.usage.totalTokens


# Reconocimiento de imagenes

### Conexión a base de datos

In [6]:
from decouple import config
import psycopg2

HOST = config('PGSQL_HOST')
USER = config('PGSQL_USER')
PASSWORD = config('PGSQL_PASSWORD')
PORT = config('PGSQL_PORT')
DB_NAME = config('PGSQL_DATABASE')

table_name = config('TABLE_NAME')
table_check = 'resumen'

def get_connection():
    connection = psycopg2.connect(
                dbname=DB_NAME,
                user=USER,
                password=PASSWORD,
                host=HOST,
                port=PORT
            )
    return connection

### CRUD

In [7]:
from psycopg2.extras import DictCursor

def create_row(infoDom, table_name=table_name):
    try:
        dictItem = infoDom.dict()

        columns = list(dictItem.keys())
        values = list(dictItem.values())

        connection = get_connection()
        with connection.cursor() as cursor:
            query = f'INSERT INTO "{table_name}" ({",".join(columns)}) VALUES ({",".join(["%s"]*len(columns))})'
            cursor.execute(query, values)
            affected_rows = cursor.rowcount
            connection.commit()
            cursor.close()
        return affected_rows
    except psycopg2.Error as e:
        raise Exception(e)

def update_row(values, table_name=table_name, columns=[], condition=None):
    try:
        connection = get_connection()

        with connection.cursor() as cursor:
            set_values = ", ".join([f"{column} = %s" for column in columns])
            condition_query = f" WHERE {condition}" if condition else ""
            query = f"UPDATE {table_name} SET {set_values}{condition_query}"
            
            cursor.execute(query, list(values.values()))  # Convertimos a lista
            connection.commit()
            cursor.close()
    except psycopg2.Error as e:
        raise Exception(e)
    
def update_rows(info_list:List[Tarjeton], table_name):
    try:
        connection = get_connection()

        with connection.cursor() as cursor:
            columns_to_update = [
                "votos_blanco", "votos_gustavo", "votos_incinerados",
                "votos_ivan", "votos_no_marcados", "votos_nulos",
                "votos_sufragantes", "votos_total", "votos_urna"
            ]

            # Prepara los nombres de columnas para el SET
            set_values = ", ".join([f"{column} = %s" for column in columns_to_update])

            # Condición para identificar la fila
            condition = (
                "departamento = %s AND municipio = %s AND puesto = %s "
                "AND zona = %s AND mesa = %s"
            )

            # Construye la consulta SQL
            query = f"""
                UPDATE {table_name}
                SET {set_values}
                WHERE {condition}
            """

            # Itera sobre la lista de objetos y ejecuta la actualización
            for info in info_list:
                # Valores para las columnas a actualizar y para la condición WHERE
                values = [
                    info.votos_blanco, info.votos_gustavo, info.votos_incinerados,
                    info.votos_ivan, info.votos_no_marcados, info.votos_nulos,
                    info.votos_sufragantes, info.votos_total, info.votos_urna,
                    info.departamento, info.municipio, info.puesto, info.zona, info.mesa
                ]

                cursor.execute(query, values)

            connection.commit()

    except Exception as e:
        print(f"Error while updating rows: {e}")


def get_all_by_columns(params, table_name=table_name):
    try:
        connection = get_connection()

        columns = list(params.keys())
        values = list(params.values())
        
        where_clause = " AND ".join([f'"{col}" = %s' for col in columns])
        with connection.cursor(cursor_factory=DictCursor) as cursor:
            query = f'SELECT * FROM "{table_name}" WHERE {where_clause}'
            cursor.execute(query, values)
            infoData = cursor.fetchall()
        connection.close()
        return infoData
    except Exception as ex:
        raise Exception(ex)
    
def get_all_rows(table_name=table_name, condition=None):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:

            condition_query = f" WHERE {condition}" if condition else ""
            query = f'SELECT * FROM "{table_name}"{condition_query}'
            cursor.execute(query)
            result = cursor.fetchall()
            cursor.close()
        return result
    except psycopg2.Error as e:
        raise Exception(e)

### Iteración de ejecución

In [8]:
import time

call_counter = 0
token_usage = 0
start_time = time.time()
MAX_CALLS_PER_MINUTE = 10
MAX_TOKENS_PER_MINUTE = 4_000_000 - 100_000

def reset_limits():
    """Resetea los contadores cada minuto."""
    global call_counter, token_usage, start_time
    call_counter = 0
    token_usage = 0
    start_time = time.time()

In [ ]:
import re
from tqdm import tqdm
from collections import defaultdict

resumen = get_all_rows(table_name=table_check, condition="completado = true")

for item in tqdm(resumen, desc="Procesando resumen"):
    item_check = Check(id=item[0], departamento=item[1], municipio=item[2], zona=item[3], completado=item[4])
    data = get_all_by_columns(table_name=table_name, params=item_check.dict(exclude={'id','completado'}))
    tarjetones = [Tarjeton(**item) for item in data]


    tarjetones_por_puesto = defaultdict(list)
    for tarjeton in tarjetones:
        tarjetones_por_puesto[tarjeton.puesto].append(tarjeton)
    
    valido = True
    for puesto, grupo_tarjetones in tarjetones_por_puesto.items():

        enlaces = [tarjeton.link for tarjeton in grupo_tarjetones]
        images_base64 = [pdf_to_image_base64(enlace) for enlace in enlaces]
        try:
            info_doms = [] # type: ignore
            for i in range(0, len(images_base64), 40):

                while True:
                    elapsed_time = time.time() - start_time
                    if elapsed_time >= 60:
                        reset_limits()
                    
                    if call_counter < MAX_CALLS_PER_MINUTE and token_usage <= MAX_TOKENS_PER_MINUTE:
                        break
                    else:
                        print("Esperando 1 segundo")
                        time.sleep(1)

                call_counter += 1
                info, tokens = process_base64_images(images_base64[i:i+40])
                token_usage += tokens
                info_doms.extend(info)

        except Exception as e:
            valido = False
            print(f"item: {item_check}.\nPuesto: {puesto}.\nToken utilizados: {token_usage}, Número de llamados: {call_counter}\nError procesando imágenes: {e}")
            continue
        for tarjeton in grupo_tarjetones:
            if isinstance(tarjeton.mesa, int):
                tarjeton_mesa_numero = int(re.search(r'\d+', tarjeton.mesa).group())
                info_dom = next((info_dom for info_dom in info_doms if info_dom.mesa == tarjeton_mesa_numero), None)
            else:
                info_dom = next((info_dom for info_dom in info_doms if info_dom.mesa.lower() == tarjeton.mesa.lower()), None)

            if info_dom:
                tarjeton.convert_from_dom(info_dom)

        update_rows(grupo_tarjetones,table_name=table_name)

    if valido:
        update_row({"completado": False}, table_name=table_check, columns=["completado"], condition=f"id = '{item_check.id}'")

Procesando resumen:   0%|          | 0/2809 [00:00<?, ?it/s]